In [ ]:
import os
import numpy as np
from algo import lse, newton
from plot import print_fittingline_totalerror, plot

# 輸入需要訊息
path = input('path = ')
name = input('name = ')
n = int(input('n = '))
Lambda = int(input('Lambda = '))

# 讀資料
x0=[]
b=[]
filepath = os.path.join(path, name)
with open(filepath) as f:
    for line in f.readlines():
        s = line.split(',')
        x0.append(float(s[0]))
        b.append(float(s[1]))

# 做矩陣A
A = []
for v in x0:
    r = []
    for i in range(n):
        r.append(v ** i)
    A.append(r)

A = np.array(A)
b = np.array(b).reshape((-1,1))

# lse
print("lse:")
parameters_lse, loss_lse = lse(A, b, n, Lambda)
print_fittingline_totalerror(n-1, parameters_lse, loss_lse)
print()

# Netwon's method
print("Newton's Method:")
parameters_newton, loss_newton = newton(A, b, n)
print_fittingline_totalerror(n-1, parameters_newton, loss_newton)

# plot
plot(x0, b, parameters_lse, parameters_newton)

In [1]:
import numpy as np

def eliminate(r1, r2, col, target=0):
    fac = (r2[col]-target) / r1[col]
    for i in range(len(r2)):
        r2[i] -= fac * r1[i]


def gauss(a): 
    # 1. 把矩陣變上三角矩陣
    for i in range(len(a)):
        if a[i][i] == 0:        # 對角線是0，和下面一列交換
            for j in range(i+1, len(a)):
                if a[i][j] != 0:
                    a[i], a[j] = a[j], a[i]
                    print(a)
                    break
            else:
                raise ValueError("Matrix is not invertible")
        
        for j in range(i+1, len(a)):
            eliminate(a[i], a[j], i)
    
    # 2. 把上三角形矩陣變對角矩陣(由下往上丟列)
    for i in range(len(a)-1, -1, -1):
        for j in range(i-1, -1, -1):
            eliminate(a[i], a[j], i)
    
    # 3. 把對角矩陣變單位矩陣
    for i in range(len(a)):
        eliminate(a[i], a[i], i, target=1)
    return a


def inverse(a):
    tmp = [[] for _ in a]
    for i, row in enumerate(a):
        assert len(row) == len(a)                             # 行、列同維
        row = list(row)                                       # array轉list
        tmp[i].extend(row + [0]*i + [1] + [0]*(len(a)-i-1))   # 補identity matrix於右邊
    gauss(tmp)
    ret = []
    for i in range(len(tmp)):
        ret.append(tmp[i][len(tmp[i])//2:])
    return np.array(ret)